Modeling 
This notebook contains our machine learning algorithm (Naive Bayes: partially implemented, logistic regression: not implemented yet, ...). It builds on the preprocessing steps (bag-of-words and tf-idf). 

In [13]:
# Standard libraries
import os
import re
import warnings
import nltk

# Data manipulation
import pandas as pd
import numpy as np

# Text processing
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.sentiment import SentimentIntensityAnalyzer
from imblearn.over_sampling import SMOTE

# Machine learning
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    StratifiedKFold,
    GridSearchCV,
)
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectKBest, chi2
from scipy.sparse import hstack
from sklearn.utils.class_weight import compute_class_weight


Naive Bayes algorithm

In [14]:
# --- File Paths Setup ---
# Base directory (go 2 levels up from /src1/)
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Define input/output directories using relative paths
input_dir = os.path.join(base_dir, "data", "intermediate")
output_dir = os.path.join(base_dir, "data", "intermediate")

In [15]:
# Naive Bayes


# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()

# Function for Text Preprocessing (Lemmatization & Cleaning)


def preprocess_text(text):
    text = text.lower()  
    text = re.sub(r"http\S+", "", text) 
    text = re.sub(r"\d+", "", text) 
    text = re.sub(r"[^a-z\s]", "", text)  
    tokens = word_tokenize(text)  
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]  
    return " ".join(lemmatized_tokens)  



# Load dataset
data_file = os.path.join(base_dir, "data", "raw", "reviews_2010.json")
df = pd.read_json(data_file, lines=True)

# Apply preprocessing to text column
df['clean_text'] = df['text'].apply(preprocess_text)

# Define Multi-Class Sentiment Labels
df['sentiment'] = df['stars'].apply(lambda x: 0 if x <= 2 else (1 if x == 3 else (2 if x == 4 else 3)))

# Define Custom Stopwords (Convert set to list!)
custom_stopwords = ["restaurant", "food", "place", "service", "menu", "great", "good", "bad", "nice", "love", "best"]


# TF-IDF Vectorization with Optimized Parameters
# TF-IDF Vectorization with Optimized Parameters
vectorizer = TfidfVectorizer(
    max_features=400000,  
    sublinear_tf=True,
    max_df=0.5,  
    min_df=3,  
    ngram_range=(1,3),  
    stop_words=custom_stopwords,  
    norm='l2'
)

X_tfidf = vectorizer.fit_transform(df['clean_text'])

# Generate Additional Features
df['review_length'] = df['clean_text'].apply(lambda x: len(x.split()))
df['num_exclamation'] = df['text'].apply(lambda x: x.count('!'))
df['avg_word_length'] = df['clean_text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if x.split() else 0)

# Sentiment Scores (Using VADER)
df['sentiment_pos'] = df['text'].apply(lambda x: sia.polarity_scores(x)['pos'])
df['sentiment_neg'] = df['text'].apply(lambda x: sia.polarity_scores(x)['neg'])
df['sentiment_neu'] = df['text'].apply(lambda x: sia.polarity_scores(x)['neu'])

# Convert additional features to a numpy array
X_additional = np.array(df[["review_length", "num_exclamation", "avg_word_length"]])

# Combine TF-IDF with additional features
X_final = hstack((X_tfidf, X_additional))  
y = df['sentiment'].values  

# Handle Class Imbalance Using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_final, y)

# Compute Class Weights to Adjust for Remaining Imbalance
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_resampled), y=y_resampled)
class_weight_dict = dict(zip(np.unique(y_resampled), class_weights))

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)


# Optimize Naive Bayes Model with Grid Search
param_grid = {'alpha': np.linspace(0.001, 2.0, 20)}  
grid_search = GridSearchCV(ComplementNB(), param_grid, cv=10, scoring='accuracy', n_jobs=-1)  
grid_search.fit(X_train, y_train)
best_alpha = grid_search.best_params_['alpha']

# Train Optimized Naive Bayes Model with Class Weights
nb_classifier = ComplementNB(alpha=best_alpha)
nb_classifier.fit(X_train, y_train)


# Make Predictions
y_pred = nb_classifier.predict(X_test)

# Evaluate Performance
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive', 'Very Positive']))


# Perform Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(nb_classifier, X_resampled, y_resampled, cv=skf, n_jobs=-1)
print(f" Stratified Cross-validation accuracy: {cv_scores.mean():.4f}")


FileNotFoundError: File m:\OneDrive\Dokumente\GitHub\techlabs-data-science-yelp\data\raw\reviews_2010.json does not exist

Multinomial Logistic Regression algorithm

In [ ]:
# Load dataset
file_path = os.path.join(input_dir, "bow_vector.csv")
df = pd.read_csv(file_path)

# Assume the first column is the text reviews (not used in ML model) and the second column is "sentiment label"
X = df.iloc[:, 2:]  # Features (numerical representation of reviews)
y = df.iloc[:, 1]  # Target (sentiment label)

# Split into train-test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train Multinomial Logistic Regression
model = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=500)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Classification report
print("Classification Report:\n", classification_report(y_test, y_pred))